<a href="https://colab.research.google.com/github/MrDegglan/9785-crypto-ml/blob/Raouf-Younes/ML_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Import
https://trello.com/b/8l8sGsy8/project-proposal

In [1]:
!pip install cryptocmd

# !pip install numpy
# !pip install pandas
# !pip install -U scikit-learn
# !pip install matplotlib

# jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


tabular vs multidimensional

pandas used for tabular data, statistics 

numpy used for multidimentional like images

In [2]:
# Library for crypto currency prediction
from cryptocmd import CmcScraper

# Dependency for pandas, provides arrays, < 50K
import numpy as np

# adds support for DataFrames, > 500K
import pandas as pd

# MinMaxScalar normalises numeric data between 0 and 1 for the purpose of ML training
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn.preprocessing import LabelEncoder

# Used for making a train and test split
from sklearn.model_selection import train_test_split

# If training new models set TRAINING to true
TRAINING = True

# If running on local machine set LOCAL to true, Neccessary for training.
LOCAL = True

# Hourly Data Source
HOURLY = False

In [3]:
if(LOCAL):
  model_simple_path = './Models/RNN_LSTM_SIMPLE/'
  model_complex_path = './Models/RNN_LSTM_COMPLEX/'
  gru_base_path = './Models/RNN_GRU_BASE/'
else:
  from google.colab import drive
  drive.mount('/content/drive')

  model_simple_path = '/content/drive/MyDrive/9785 Colab Notebooks/Ml_Models/RNN_LSTM_SIMPLE/'
  model_complex_path = '/content/drive/MyDrive/9785 Colab Notebooks/Ml_Models/RNN_LSTM_COMPLEX/'
  gru_base_path = '/content/drive/MyDrive/9785 Colab Notebooks/Ml_Models/RNN_GRU_BASE/'


# Data Download and preperation

In [4]:
# Data Import
if(HOURLY):
  data = pd.read_csv('/content/drive/MyDrive/9785 Colab Notebooks/Data/Gemini_BTCUSD_1h.csv',skiprows=1)
  data.columns = [x.strip().lower().replace(' ','_') for x in data.columns]
  display(data)
else:
  # ==============================================================================
  # Scraper is initialized, symbol, start and end of download are included
  scraper = CmcScraper('BTC', '28-04-2013', '24-08-2022')
  # Transform collected data into a dataframe
  data = scraper.get_dataframe()
  data.columns = [x.strip().lower().replace(' ','_') for x in data.columns]
  display(data)

,date,open,high,low,close,volume,market_cap
0,2022-08-24,21526.455593,21783.075723,21195.006695,21395.020071,3.196225e+10,4.093085e+11
1,2022-08-23,21401.045670,21646.203762,20955.138156,21528.087372,3.187828e+10,4.118332e+11
2,2022-08-22,21531.463849,21531.463849,20939.184173,21398.908350,3.166650e+10,4.093401e+11
3,2022-08-21,21160.392783,21668.845908,21103.196808,21534.121231,2.310231e+10,4.119067e+11
4,2022-08-20,20872.842174,21350.806299,20856.731336,21166.060396,2.759567e+10,4.048481e+11
...,...,...,...,...,...,...,...
3401,2013-05-02,116.379997,125.599998,92.281898,105.209999,0.000000e+00,1.168517e+09
3402,2013-05-01,139.000000,139.889999,107.720001,116.989998,0.000000e+00,1.298955e+09
3403,2013-04-30,144.000000,146.929993,134.050003,139.000000,0.000000e+00,1.542813e+09
3404,2013-04-29,134.444000,147.488007,134.000000,144.539993,0.000000e+00,1.603769e+09


In [5]:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')#.astype(int) / 10**9
data['hour'] = data['date'].dt.hour
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data = data.drop(['date','unix','symbol'], axis=1,errors='ignore')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  display(data.head(5))
  print(len(data))

,open,high,low,close,volume,market_cap,hour,day,month,year
0,21526.455593,21783.075723,21195.006695,21395.020071,3.196225e+10,4.093085e+11,0,24,8,2022
1,21401.045670,21646.203762,20955.138156,21528.087372,3.187828e+10,4.118332e+11,0,23,8,2022
2,21531.463849,21531.463849,20939.184173,21398.908350,3.166650e+10,4.093401e+11,0,22,8,2022
3,21160.392783,21668.845908,21103.196808,21534.121231,2.310231e+10,4.119067e+11,0,21,8,2022
4,20872.842174,21350.806299,20856.731336,21166.060396,2.759567e+10,4.048481e+11,0,20,8,2022


3406


In [6]:
# Need to make all data in thousands USD, Volume ATM is in millions or billions

scaler = MinMaxScaler(feature_range=(0,1))
normalised_data = scaler.fit_transform(data)
type(normalised_data),normalised_data.shape,normalised_data[0][3],scaler.inverse_transform(normalised_data[0:1])[0][3],data.iloc[0:1,3:4]

(numpy.ndarray,
 (3406, 10),
 0.3159569613479871,
 21395.02007149252,
           close
 0  21395.020071)

In [7]:
x_train = []
y_train = []

# Looping over everyday starting from day number 60
for i in range (30, normalised_data.shape[0]):
    # the ,0 in the second index means to access the first coloumn in that row otherwise we will gettraining_set_scaled[i,0]
    x_train.append(normalised_data[i-30:i,0:10]) # data of days 0 to 59 as upper bound :i,0] is not inclusive

    y_train.append(normalised_data[i,3:4]) # data of the 60th day (the true result)

# taking the batches of data and transforming it into a numpy array instead of a list
x_train, y_train = np.array(x_train),np.array(y_train)

In [8]:
X_train, X_test = x_train[:-480], x_train[-480:]
Y_train, Y_test = y_train[:-480], y_train[-480:]

In [9]:
for i in range(0,len(x_train)):
  #print(i)
  for j in range(X_train.shape[1]):
    if(x_train[i][j][3] == y_train[0]):
      print(i,j)

1 29
2 28
3 27
4 26
5 25
6 24
7 23
8 22
9 21
10 20
11 19
12 18
13 17
14 16
15 15
16 14
17 13
18 12
19 11
20 10
21 9
22 8
23 7
24 6
25 5
26 4
27 3
28 2
29 1
30 0


# Build Tensorflow Model

## Standard Imports

In [10]:
# Importing Tensorflow
import tensorflow as tf

# Declares a base class called sequential to add model layers on 
from tensorflow.keras.models import Sequential

# Dense layer is final output 1d layer
# Dropout layer protects against overfitting 
from tensorflow.keras.layers import Dense, Dropout

# RNN LSTM layer
from tensorflow.keras.layers import LSTM,GRU

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Mounting google drive for save
# Moved up

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Building RNN - LSTM

In [ ]:
model_simple=Sequential()
model_simple.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
model_simple.add(LSTM(50,return_sequences=True))
model_simple.add(LSTM(50))
model_simple.add(Dense(1))
model_simple.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model_complex=Sequential()
model_complex.add(LSTM(256,return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
model_complex.add(Dropout(rate=0.2))
model_complex.add(LSTM(512,return_sequences=True))
model_complex.add(Dropout(rate=0.4))
model_complex.add(LSTM(512,return_sequences=True))
model_complex.add(Dropout(rate=0.4))
model_complex.add(LSTM(256))
model_complex.add(Dense(1))
model_complex.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
if(TRAINING):
  callbacks_simple = [EarlyStopping(monitor='val_loss', patience=50),
              ModelCheckpoint(model_simple_path, monitor='loss', save_best_only=True, mode='min')]
  callbacks_complex = [EarlyStopping(monitor='val_loss', patience=50),
              ModelCheckpoint(model_complex_path, monitor='loss', save_best_only=True, mode='min')]
  model_simple.fit(x_train, y_train,validation_split=0.2,epochs=1000,batch_size=32,verbose=1,callbacks=callbacks_simple)
  model_complex.fit(x_train, y_train,validation_split=0.2,epochs=1200,batch_size=64,verbose=1,callbacks=callbacks_complex)
else:
  model_simple = model_simple.load_weights(model_simple_path)
  model_complex = model_complex.load_weights(model_complex_path)

In [ ]:
predictions_simple = model_simple.predict(X_test)
predictions_complex = model_complex.predict(X_test)

## RNN - GRU

In [ ]:
GRU_Base=Sequential()
GRU_Base.add(GRU(50,return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
GRU_Base.add(GRU(50,return_sequences=True))
GRU_Base.add(GRU(50))
GRU_Base.add(Dense(1))
GRU_Base.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
callbacks_base = [EarlyStopping(monitor='val_loss', patience=150),
            ModelCheckpoint(gru_base_path, monitor='loss', save_best_only=True, mode='min')]
GRU_Base.fit(x_train, y_train,validation_split=0.2,epochs=1000,batch_size=32,verbose=1,callbacks=callbacks_base)

In [ ]:
GRU_PRED_BASE = GRU_Base.predict(X_test)

## Autoregressive Forecaster

# Visualise the model predictions

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.figure(figsize=(24,12))
plt.title('Forecasting Closing price of bitcoin based on previous 30 days')
plt.xlabel('Time', fontsize=14) # Fix Timeing
plt.ylabel('Bitcoin Price (AUD)', fontsize=18)
plt.plot(predictions_simple,color='blue')
plt.plot(predictions_complex,color='yellow')
plt.plot(GRU_PRED_BASE,color='pink')
plt.plot(Y_test,color='green')
plt.legend(['Forecast_Simple','Forecast_Complex','Gru Base', 'Actual'])
plt.show()

# Resources

https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21